In [55]:

from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [56]:
!git clone https://github.com/kakao/khaiii.git

!pip install cmake

!mkdir build

!cd build && cmake /content/khaiii

!cd /content/build/ && make all

!cd /content/build/ && make resource

!cd /content/build && make install

!cd /content/build && make package_python

!pip install /content/build/package_python
!pip install koco

fatal: destination path 'khaiii' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
-- [khaiii] fused multiply add option enabled
-- [hunter] Calculating Toolchain-SHA1
-- [hunter] Calculating Config-SHA1
-- [hunter] HUNTER_ROOT: /root/.hunter
-- [hunter] [ Hunter-ID: 70287b1 | Toolchain-ID: 02ccb06 | Config-ID: dffbc08 ]
-- [hunter] BOOST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.68.0-p1)
-- Boost version: 1.68.0
-- [hunter] CXXOPTS_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 2.1.1-pre)
-- [hunter] EIGEN_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.5)
-- [hunter] FMT_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 4.1.0)
-- [hunter] GTEST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.8.0-hunter-p11)
-- [hunter] NLOHMANN_JSON_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.0)
-- [hunter] SPDLOG_ROOT: /root/.hunte

In [57]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
#주요 참고 PyTorch로 시작하는 딥 러닝 입문, 유원준
from torchtext import data  
import urllib.request
import pandas as pd
import random
import koco

In [58]:
from khaiii import KhaiiiApi
api = KhaiiiApi()


In [59]:
train_dev = koco.load_dataset('korean-hate-speech', mode='train_dev')

In [60]:
import csv

with open('/content/gdrive/My Drive/datas/hate.csv', 'w', newline='') as csvfile:
    fieldnames = ['document', 'lable']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for train in train_dev['train']:
        lable = 1
        if train['hate'] != 'none':
            lable = 0
        writer.writerow({'document':train['comments'],'lable':lable})


In [61]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [62]:
#데이터 전처리 후에 남은 문자들이 전부 빈칸일 때 제거하기 위함
def isAll0(x):
    if type(x) == float:
        return x
    elif len(x) == x.count(' '):
        return ''
    else:
        return x


In [63]:
'''
#데이터 가져와서
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")
'''
#읽고
all_df = pd.read_csv('/content/gdrive/My Drive/datas/hate.csv')
train_df = all_df[:int(len(all_df)*0.9)]
test_df = all_df[int(len(all_df)*0.9):]

#학습 데이터 전처리, 영어 특수문자 제거, 공백 제거
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df['document'] = train_df['document'].apply(isAll0)
train_df['document'].replace('', np.nan, inplace=True)

#테스트 데이터 전처리
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['document'] = test_df['document'].apply(isAll0)
test_df['document'].replace('', np.nan, inplace=True)

#중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)

#Null 제거
train_df = train_df.dropna(how = 'any')
test_df = test_df.dropna(how = 'any')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [64]:
print(len(train_df))
print(len(test_df))

7104
790


In [65]:
#불용어
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [66]:
train_x = []

In [67]:
#학습 데이터 형태소로 분리, 문장 시작, 끝 추가
j = 0
for i, row in train_df.iterrows():
    val = row['document']
    j += 1
    if j % 10000 == 0:
        print(i,"/","143660")

    temp = ['CLS']
    for word in api.analyze(val):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                temp.append(morph.lex)
    temp.append('SEP')
    train_x.append(temp)


In [68]:
test_x = []

In [69]:
#테스트 데이터 형태소로 분리
j = 0
for i, row in test_df.iterrows():
    val = row['document']
    j += 1
    if i % 10000 == 0:
        print(j,"/","48000")
    temp = ['CLS']
    
    for word in api.analyze(val):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                temp.append(morph.lex)
    temp.append('SEP')
    test_x.append(temp)


In [70]:
vocab_size = 6000

#형태소를 정수로 인코딩
tokenizer = Tokenizer(vocab_size,oov_token = 'OOV')
with open('/content/gdrive/My Drive/GRUmodel/tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [71]:
train_y = np.array(train_df['lable'])
test_y = np.array(test_df['lable'])

In [72]:
# 제거 후 빈칸만 남은 거 또한 제거
drop_train = [index for index, sentence in enumerate(train_x) if len(sentence) < 1]
drop_test = [index for index, sentence in enumerate(test_x) if len(sentence) < 1]

In [73]:
train_x = np.delete(train_x, drop_train, axis=0)
train_y = np.delete(train_y, drop_train, axis=0)
print(len(train_x))
print(len(train_y))

7104
7104


In [74]:
test_x = np.delete(test_x, drop_test, axis=0)
test_y = np.delete(test_y, drop_test, axis=0)
print(len(test_x))
print(len(test_y))

790
790


In [75]:
#패딩
pad_len = 30
train_x = pad_sequences(train_x, maxlen = pad_len,padding='post')
test_x = pad_sequences(test_x, maxlen = pad_len,padding='post')

In [76]:
#데이터 셋 간단하게
class nlp_dataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [77]:
#베이스 라인
class grubase(nn.Module):
    def __init__(self, embed_dim, vocab_size, hidden_dim, num_layers, batch_size, dropout):
        super(grubase, self).__init__()
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.relu = nn.ReLU()
        self.batch_size = batch_size
        self.sigmoid = nn.Sigmoid()


        self.norm = nn.BatchNorm1d(self.batch_size)
        self.embed = nn.Embedding(self.vocab_size,self.embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.gru = nn.GRU(self.embed_dim, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers,batch_first=True)

        self.mlp1 = nn.Linear(self.hidden_dim,1)

    def forward(self,x):
        x = self.embed(x)
        x = self.dropout(x)

        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        x = x[:,-1,:]
        #x = torch.cat((x[:,0,:],x[:,-1,:]),dim=-1)

        x = self.dropout(x)
        x = self.mlp1(x)
        #x = self.sigmoid(x).squeeze()
        return x.squeeze()


In [78]:
# 튜닝
class grumodel(nn.Module):
    def __init__(self, embed_dim, vocab_size, hidden_dim, num_layers, batch_size, dropout):
        super(grumodel, self).__init__()
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.relu = nn.ReLU()
        self.batch_size = batch_size
        self.sigmoid = nn.Sigmoid()


        self.norm = nn.BatchNorm1d(self.batch_size)
        self.embed = nn.Embedding(self.vocab_size,self.embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.gru = nn.GRU(self.embed_dim, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)

        self.mlp1 = nn.Linear(self.hidden_dim*4,self.hidden_dim)
        self.mlp2 = nn.Linear(self.hidden_dim,self.hidden_dim//4)
        self.mlp3 = nn.Linear(self.hidden_dim//4,1)

    def forward(self,x):
        x = self.embed(x)
        x = self.dropout(x)

        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        x = torch.cat((x[:,0,:],x[:,-1,:]),dim=-1)

        x = self.dropout(x)
        x = self.mlp1(x)
        x = self.relu(x)
        x = self.mlp2(x)
        x = self.relu(x)
        x = self.mlp3(x)
        return x.squeeze()


In [79]:
#학습
def train(model, optimizer, loss_function,train_loader,DEVICE):
    model.train()
    for batch in train_loader:
        x, y = batch
        x = x.long().to(DEVICE)
        y = y.long().to(DEVICE)
        optimizer.zero_grad()

        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        loss.backward()
        optimizer.step()

In [80]:
# 성능검증 용, F1 스코어, 정확도
def getF1(y_pred,y,threshold=0.5):
    
    yp = [1 if x > threshold else 0 for x in y_pred]

    pp = 0
    pf = 0
    fp = 0
    ff = 0
    for i in range(len(y)):
        if y[i] > threshold:
            if yp[i] > threshold: pp += 1
            else: pf += 1
        else:
            if yp[i] < threshold: ff += 1
            else: fp += 1

    precision = pp / (pp + fp + 1e-5) 
    recall = pp / (pp + ff + 1e-5)
    F1 = 2 * precision * recall / (precision + recall + 1e-5)
    acc = (pp + ff) / (len(y) + 1e-5)
    return F1, acc

In [81]:
# 검증
def evaluate(model, val_loader, loss_function, DEVICE, batch_size, threshold):
    """evaluate model"""
    model.eval()
    total_loss = 0
    total_f1 = 0
    total_acc = 0
    for batch in val_loader:
        x, y = batch
        x = x.long().to(DEVICE)
        y = y.long().to(DEVICE)
        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        f1, acc = getF1(y_pred,y,threshold)
        total_f1 += f1
        total_acc += acc
        total_loss += loss.item()

    size = len(val_loader.dataset) / batch_size
    avg_loss = total_loss / size
    avg_f1 = total_f1 / size
    avg_acc = total_acc / size
    return avg_loss, avg_f1, avg_acc

In [82]:
device = torch.device("cuda")

batch_size = 256
embed_dim = 368
hidden_dim = 512
dropout = 0.7
layers = 1

model = grumodel(embed_dim,vocab_size,hidden_dim,layers,batch_size,dropout)
modelPath = '/content/gdrive/My Drive/GRUmodel/Khaiii_gru_model.pt'
model.load_state_dict(torch.load(modelPath))
model.to(device)
loss = nn.BCEWithLogitsLoss(pos_weight = 1.1 * torch.ones([1])).to(device)
lr = 0.001
threshold = 0.5

EPOCHS = 20
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [83]:
train_dataset = nlp_dataset(train_x,train_y)
test_dataset = nlp_dataset(test_x,test_y)

train_loader = DataLoader(train_dataset,batch_size,True,drop_last=True)
val_loader = DataLoader(test_dataset,batch_size,True,drop_last=True)

In [84]:
print(train_dataset[0])

(array([   2,  920,    1, 3080,   21,   17,    5,    1,    1,    8,    1,
          4,   15,  648,   11,   25, 1597,  399,  316,    1,  164,   33,
          3,    0,    0,    0,    0,    0,    0,    0], dtype=int32), 0)


In [85]:
best_val_f1 = 0
for e in range(1, EPOCHS+1):
    train(model, optimizer, loss,train_loader,device)
    val_loss,val_f1,val_acc = evaluate(model, val_loader, loss, device,batch_size,threshold)

    print("[Epoch: %d] val loss : %1.5f    val acc :%4.3f    F1 :%4.3f" % (e, val_loss, val_acc,val_f1))

    # F1 성능 지표로 저장
    if not best_val_f1 or val_f1 > best_val_f1:
        print("Best saved")
        torch.save(model.state_dict(), '/content/gdrive/My Drive/GRUmodel/Khaiii_best_model.pt')
        best_val_f1 = val_f1

[Epoch: 1] val loss : 0.60340    val acc :0.654    F1 :0.367
Best saved
[Epoch: 2] val loss : 0.58831    val acc :0.668    F1 :0.404
Best saved
[Epoch: 3] val loss : 0.57885    val acc :0.680    F1 :0.418
Best saved
[Epoch: 4] val loss : 0.57814    val acc :0.682    F1 :0.415
[Epoch: 5] val loss : 0.56246    val acc :0.697    F1 :0.452
Best saved
[Epoch: 6] val loss : 0.56873    val acc :0.689    F1 :0.441
[Epoch: 7] val loss : 0.55995    val acc :0.687    F1 :0.442
[Epoch: 8] val loss : 0.55670    val acc :0.692    F1 :0.440
[Epoch: 9] val loss : 0.55933    val acc :0.691    F1 :0.449
[Epoch: 10] val loss : 0.55930    val acc :0.692    F1 :0.445
[Epoch: 11] val loss : 0.56514    val acc :0.700    F1 :0.446
[Epoch: 12] val loss : 0.57542    val acc :0.700    F1 :0.479
Best saved
[Epoch: 13] val loss : 0.58104    val acc :0.699    F1 :0.463
[Epoch: 14] val loss : 0.59262    val acc :0.700    F1 :0.452
[Epoch: 15] val loss : 0.58810    val acc :0.691    F1 :0.457
[Epoch: 16] val loss : 0

In [88]:
model.eval()
def predict(original_sentence):
    sentence = ['CLS']
    for word in api.analyze(original_sentence):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                    sentence.append(morph.lex)
    sentence.append('SEP')
    x = tokenizer.texts_to_sequences([sentence])
    x = pad_sequences(x, maxlen = pad_len,padding='post') # 패딩
    x = torch.tensor(x).to(device).long()
    
    y = model(x)
    if(y > 0):
        return 'P'
    else:
        return 'B'

In [92]:
print(predict("개소리 오지네"))
print(predict("안녕?"))
print(predict("아니 씨발"))
print(predict("선동 ㄴㄴ"))
print(predict("ㅁㅊ ㅅㅂ"))
print(predict("ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ"))
print(predict("존나 웃기네"))
print(predict("안녕하세요"))
print(predict("개소리를 길게도 쓰셨네요"))
print(predict("나가 뒤지세요"))

B
B
P
P
B
P
P
P
P
P


In [86]:
'''
import json
tokenizer_json = tokenizer.to_json()
with open('/content/gdrive/My Drive/GRUmodel/tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))
'''

"\nimport json\ntokenizer_json = tokenizer.to_json()\nwith open('/content/gdrive/My Drive/GRUmodel/tokenizer.json', 'w', encoding='utf-8') as f:\n    f.write(json.dumps(tokenizer_json, ensure_ascii=False))\n"